In [11]:
# Import necessary PyTorch libraries
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
# Additional libraries for visualization and utilities
import matplotlib.pyplot as plt
import numpy as np

In [12]:
def get_device():
    """Selects the best available device for PyTorch computations.

    Returns:
        torch.device: The selected device.
    """

    if torch.cuda.is_available():
        return torch.device('cuda')
    elif torch.backends.mps.is_available():
        return torch.device('mps')
    else:
        return torch.device('cpu')

# device = get_device()
device = 'cpu'
print(f"using device: {device}")

using device: cpu


In [13]:
# Import the adapted Echo noise functions
from echo import echo_sample,echo_loss

In [14]:
from torchvision import datasets
from torch.utils.data import DataLoader, random_split
from torchvision.transforms import Compose, ToTensor, Normalize, Grayscale

# Define transformations: Convert to grayscale, resize if needed, and normalize the data
transform = Compose([
    Grayscale(num_output_channels=1),  # Convert to grayscale
    ToTensor(),
    Normalize((0.5,), (0.5,))  # Normalize with single channel
])

# Load the CIFAR-10 dataset
dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

# Splitting dataset into training and validation sets
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create DataLoader instances
train_loader = DataLoader(train_dataset, batch_size=100, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=100, shuffle=False)

print("Data loaders created for training and validation.")

Files already downloaded and verified
Data loaders created for training and validation.


In [15]:
from segmentation_models_pytorch import Unet

class Encoder(nn.Module):
    def __init__(self, input_shape, latent_dims):
        super(Encoder, self).__init__()
        self.input_shape = input_shape
        self.latent_dims = latent_dims

        self.unet = Unet(
            encoder_name="resnet18",
            encoder_depth=3,
            encoder_weights=None,
            decoder_use_batchnorm=True,
            decoder_channels=(latent_dims[2], latent_dims[1], latent_dims[0]),
            decoder_attention_type=None,
            in_channels=input_shape[0],
            classes=latent_dims[-1],
            activation=None,
        )

        # Output layers
        self.out_mean = nn.Conv2d(latent_dims[-1], input_shape[0], kernel_size=1)
        self.out_log_var = nn.Conv2d(latent_dims[-1], input_shape[0], kernel_size=1)

    def forward(self, x):
        x = self.unet(x)
        f_x = torch.tanh(self.out_mean(x))
        log_var = torch.sigmoid(self.out_log_var(x))
        return f_x, log_var

In [16]:
def get_timestep_embedding(timesteps, embedding_dim):
    """
    This matches the implementation in Denoising Diffusion Probabilistic Models:
    From Fairseq.
    Build sinusoidal embeddings.
    This matches the implementation in tensor2tensor, but differs slightly
    from the description in Section 3.5 of "Attention Is All You Need".
    """
    assert len(timesteps.shape) == 1

    half_dim = embedding_dim // 2
    emb = math.log(10000) / (half_dim - 1)
    emb = torch.exp(torch.arange(half_dim, dtype=torch.float32) * -emb)
    emb = emb.to(device=timesteps.device)
    emb = timesteps.float()[:, None] * emb[None, :]
    emb = torch.cat([torch.sin(emb), torch.cos(emb)], dim=1)
    if embedding_dim % 2 == 1:  # zero pad
        emb = torch.nn.functional.pad(emb, (0,1,0,0))
    return emb

In [17]:
class Decoder(nn.Module):
    def __init__(self, latent_dims, output_shape, timestep_dim=128):
        super(Decoder, self).__init__()
        self.latent_dims = latent_dims
        self.output_shape = output_shape
        self.timestep_dim = timestep_dim

        self.conv1 = nn.Conv2d(1, latent_dims[0], kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(latent_dims[0], latent_dims[1], kernel_size=3, stride=2, padding=1)
        self.conv3 = nn.Conv2d(latent_dims[1], latent_dims[2], kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(latent_dims[2], latent_dims[3], kernel_size=3, stride=2, padding=1)
        self.conv5 = nn.Conv2d(latent_dims[3], latent_dims[4], kernel_size=3, stride=1, padding=1)

        self.timestep_mlp = nn.Sequential(
            nn.Linear(timestep_dim, latent_dims[4]),
            nn.ReLU(),
            nn.Linear(latent_dims[4], latent_dims[4]),
        )

        self.deconv1 = nn.ConvTranspose2d(latent_dims[4] * 2, latent_dims[3], kernel_size=3, stride=1, padding=1)
        self.deconv2 = nn.ConvTranspose2d(latent_dims[3], latent_dims[2], kernel_size=4, stride=2, padding=1)
        self.deconv3 = nn.ConvTranspose2d(latent_dims[2], latent_dims[1], kernel_size=3, stride=1, padding=1)
        self.deconv4 = nn.ConvTranspose2d(latent_dims[1], latent_dims[0], kernel_size=4, stride=2, padding=1)
        self.deconv5 = nn.ConvTranspose2d(latent_dims[0], output_shape[0], kernel_size=3, stride=1, padding=1)

    def forward(self, x, t):
        timestep_emb = get_timestep_embedding(t, self.timestep_dim)
        timestep_emb = self.timestep_mlp(timestep_emb).detach()

        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x = torch.relu(self.conv3(x))
        x = torch.relu(self.conv4(x))
        x = torch.relu(self.conv5(x))

        x = torch.cat([x, timestep_emb[:, :, None, None].repeat(1, 1, x.shape[2], x.shape[3])], dim=1)
        del timestep_emb
        torch.mps.empty_cache()

        x = torch.relu(self.deconv1(x))
        x = torch.relu(self.deconv2(x))
        x = torch.relu(self.deconv3(x))
        x = torch.relu(self.deconv4(x))
        x = torch.sigmoid(self.deconv5(x))

        return x

In [18]:
class EchoModel(nn.Module):
    def __init__(self, input_shape, latent_dims, output_shape, T=1000, batch_size=100):
        super(EchoModel, self).__init__()
        self.input_shape = input_shape
        self.latent_dims = latent_dims
        self.output_shape = output_shape
        self.T = T
        self.batch_size = batch_size

        self.encoder = Encoder(input_shape, latent_dims)
        self.decoder = Decoder(latent_dims, output_shape)

        # Define the noise schedule
        self.alpha = self.create_noise_schedule(T)

    def create_noise_schedule(self, T):
        alpha = torch.linspace(0.9999, 1e-5, T)
        return alpha

    def forward(self, x):
        #Calculate f_x and S_x
        f_x, sx_matrix = self.encoder(x)
        mi_loss = echo_loss(sx_matrix)
        print(mi_loss.item())
        assert(False)
        
        torch.mps.empty_cache()
        
        #Calculate epsilon in a detached way
        epsilon = echo_sample((f_x, sx_matrix)).detach()

        torch.mps.empty_cache()

        #Calculate echo output z 
        z = f_x + sx_matrix * epsilon

        del epsilon 
        
        torch.mps.empty_cache()
        
        # Retrieve noise scheduler alpha_T
        alpha_T = self.alpha[-1]

        # Calculate square root alphas
        sqrt_alpha_T = torch.sqrt(alpha_T)
        sqrt_one_minus_alpha_T = torch.sqrt(1 - alpha_T)
        
        # Perform the weighted sum
        x_T = sqrt_alpha_T * x + sqrt_one_minus_alpha_T * z


        # Perform the reconstruction process using Algorithm 2
        reconstructed_x = self.reconstruct(x_T)
        torch.mps.empty_cache()
        return reconstructed_x, f_x, sx_matrix

    def reconstruct(self, x_t):
        x_s = x_t
        x_0_hat = x_t
        for s in range(self.T-1, 0, -1):
            t = torch.tensor([s] * x_t.size(0), dtype=torch.long).to(x_t.device)
            sqrt_alpha_s = torch.sqrt(self.alpha[s])
            sqrt_one_minus_alpha_s = torch.sqrt(1 - self.alpha[s])

            # Estimate the original image using the decoder
            x_0_hat = self.decoder(x_s, t)

            # Calculate the estimated noise using Eq. (3)
            z_hat = (x_s - sqrt_alpha_s * x_0_hat) / sqrt_one_minus_alpha_s

            # Calculate D(x_0_hat, s) and D(x_0_hat, s-1) using Eq. (5) and (6)
            D_x_0_hat_s = sqrt_alpha_s * x_0_hat + sqrt_one_minus_alpha_s * z_hat
            D_x_0_hat_s_minus_1 = torch.sqrt(self.alpha[s-1]) * x_0_hat + torch.sqrt(1 - self.alpha[s-1]) * z_hat

            # Update x_s using Eq. (7)
            x_s = x_s - D_x_0_hat_s + D_x_0_hat_s_minus_1

        return x_0_hat

In [19]:
import time  # Importing time to log the duration

import os

def save_checkpoint(epoch, model, optimizer, filename="checkpoint.pth"):
    """Saves the model and optimizer state at the specified path."""
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict()
    }, filename)
    print(f"Checkpoint saved at epoch {epoch} to {filename}")


def train(model, optimizer, train_loader, device, num_epochs, loss_weights, accumulation_steps=2):
    model.train()  # Ensure the model is in training mode

    for epoch in range(num_epochs):
        epoch_loss = 0.0
        # optimizer.zero_grad()  # Reset gradients at the start of each epoch

        print(f"Starting epoch {epoch + 1}/{num_epochs}")
        for batch_idx, (data, _) in enumerate(train_loader):
            batch_start_time = time.time()  # Start time for the batch
            data = data.to(device)  # Move data to the appropriate device (GPU/CPU)

            # Forward pass
            reconstructed_x, f_x, sx_matrix = model(data)
            reconstruction_loss = F.mse_loss(reconstructed_x, data)
            del reconstructed_x
            torch.mps.epty_cache()
            # mi_penalty = echo_loss(sx_matrix)
            total_loss = loss_weights['reconstruction'] * reconstruction_loss

            print(f"Total loss: {total_loss.item()}")

            # Backward pass
            total_loss.backward()

            # Update parameters every 'accumulation_steps' batches
            if (batch_idx + 1) % accumulation_steps == 0 or (batch_idx + 1) == len(train_loader):
                optimizer.step()  # Perform a single optimization step
                optimizer.zero_grad(set_to_none=True)  # Reset gradients after updating

            print(f"\tBatch {batch_idx + 1}/{len(train_loader)} processed in {time.time() - batch_start_time:.2f} seconds.")

            # Accumulate epoch loss
            epoch_loss += total_loss.item()

        avg_loss = epoch_loss / len(train_loader)
        print(f"Epoch [{epoch + 1}/{num_epochs}] completed in {time.time() - epoch_start_time:.2f} seconds, Avg Loss: {avg_loss}")

    return model

In [22]:
# Define the input shape, latent dimensions, and output shape
input_shape = (1, 32, 32)  # Shape for grayscale CIFAR-10 (1 channel, 32x32 images)
latent_dims = [32, 64, 128, 256, 512]  # Updated latent dimensions
output_shape = (1, 32, 32)  # Shape for grayscale CIFAR-10

# Create an instance of the EchoModel
model = EchoModel(input_shape, latent_dims, output_shape).to(device)

# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Define the number of epochs and loss weights
num_epochs = 100
loss_weights = {'reconstruction': 1.0, 'mi_penalty': 0.0}  # Adjust the weights as needed

# Train the model
trained_model = train(model, optimizer, train_loader, device, num_epochs, loss_weights)

Starting epoch 1/100
-93.81854248046875


AssertionError: 